In [ ]:
import numpy as np
import torch
import einops
import math
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np
import numba as nb
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F

from src.mamba import Mamba, MambaConfig 
from sbi import utils as utils
from sbi import analysis as analysis
from sbi.inference.base import infer
from sbi.inference import SNPE, prepare_for_sbi, simulate_for_sbi
from sbi import analysis, utils
from sbi.inference import SNPE, simulate_for_sbi
from sbi.utils.user_input_checks import (
    check_sbi_inputs,
    process_prior,
    process_simulator,
)
# import required modules
from sbi.utils.get_nn_models import posterior_nn
seed = 0 
torch.manual_seed(seed) 
from src.temporal_encoders import ResidualTemporalBlock, Residual, PreNorm, LinearAttention, Downsample1d, Conv1dBlock

In [ ]:
with open('your_path.pkl', 'rb') as f:
    posterior_Lan = torch.load(f)

with open('your_path.pkl', 'rb') as f:
    posterior_BM = torch.load(f)

For Langevin posterior:

In [ ]:
def simulate_brownian_motion(num_steps=1000, dt = 5e-06, replica_num=1, D=10, x0=np.array([-1.5], dtype=np.float64), save_every=1, k=3):
    num_steps = int(num_steps)
    save_every = int(save_every)

    N = num_steps

    diffusion_coeffs = np.full(replica_num, D)
    Ax = diffusion_coeffs * dt
    Bx = np.sqrt(2 * Ax)

    x = np.zeros((num_steps//save_every, replica_num, 1), dtype=np.float64)
    xold = np.tile(x0, (replica_num, 1))

    for i in range(1, N):
        # forces evaluation
        Fx = -k * xold

        # Drawing noise
        gx = np.random.standard_normal(size=(replica_num, 1))

        # integration
        xnew = xold + Ax[:, None] * Fx + Bx[:, None] * gx

        if (i % save_every) == 0:
            x[i // save_every] = xnew

        xold = xnew

    return x

def Brownian_Motion_simulator1(params):
    params = np.array(params.cpu(), dtype=np.float64)
    x = simulate_brownian_motion(num_steps=1000, dt=5e-06, replica_num=1, D=10**params[0], x0=np.array([0], dtype=np.float64), save_every=1, k=3)
    return torch.tensor(x, dtype=torch.float32).unsqueeze(0)

In [ ]:
true_params = torch.tensor([1.5], dtype=torch.float32)
observation = Brownian_Motion_simulator1(true_params).to('cuda').squeeze(3)
observation.shape

In [ ]:
samples = posterior_Lan.sample((10000,), x=observation, show_progress_bars=False).cpu()

_ = analysis.pairplot(
    samples,
    limits=[mass_limits, nux_limits],
    figsize=(10, 10),
    points_offdiag={'markersize': 6},
    labels=['$\log(m)$', '$\log(\\gamma)$']
)

In [ ]:
posterior_samples = samples.cpu().numpy()
m = posterior_samples[:, 0]
gamma = posterior_samples[:, 1]

m_mean = m.mean()
gamma_mean = gamma.mean()

print(f"Mean of log(m): {m_mean}")
print(f"Mean of log(γ): {gamma_mean}")

For Brownian Posterior:

In [ ]:
@nb.jit(nopython=True, fastmath=True)
def force(x, k=3):
    Fx = -k * x
    return Fx

@nb.jit(nopython=True, fastmath=True)
def langevin_integrator(
    x0=-1.5,
    vx0=0.0,
    mx=1,
    nux=1,
    k=3,
    N=1000,
    dt=5e-5,
    fs=1
):

    beta = 1 # by default we set the temperature to 1 to simplify the calculation
    invmass = 1.0 / mx
    sigma = np.sqrt(2.0 * nux / (beta * mx))
    b1 = 1.0 - 0.5 * dt * nux + 0.125 * (dt ** 2) * (nux ** 2)
    b2 = 0.5 * dt - 0.125 * nux * dt ** 2
    s3 = np.sqrt(3.0)
    sdt3 = sigma * np.sqrt(dt ** 3)
    sdt = sigma * np.sqrt(dt)

    n = int(N / fs)
    x = np.zeros(n)
    xi = np.random.standard_normal(size=n)
    eta = np.random.standard_normal(size=n)
    x[0] = x0
    vx = vx0
    xold = x0
    Fx = force(x0, k=k) * invmass

    for i in range(1, N):

        # Drawing noise
        if i%n == 0:
            xi = np.random.standard_normal(size=n)
            eta = np.random.standard_normal(size=n)

        _xi = xi[i%n]
        _eta = eta[i%n]

        n1 = 0.5 * sdt * _xi
        n3 = 0.5 * sdt3 * _eta / s3
        n4 = sdt3 * (0.125 * _xi + 0.25 * _eta / s3)
        n5 = n1 - nux * n4

        vx = vx * b1 + Fx * b2 + n5
        xnew = xold + dt * vx + n3

        Fx = force(xnew, k=k) * invmass
        vx = vx * b1 + Fx * b2 + n5

        if (i % fs) == 0:
            x[int(i / fs)] = xnew

        xold = xnew

    return x

def Brownian_Motion_simulator1(params):
    params = np.array(params.cpu(), dtype=np.float64)
    x = langevin_integrator(
    x0=0,
    vx0=0.0,
    mx= 10**params[0],
    nux=10**params[1],
    k=3,
    N=1000,
    dt=5e-5,
    fs=1
)
    return torch.tensor(x, dtype=torch.float32).unsqueeze(-1)

In [ ]:
true_params = torch.tensor([-1,  2], dtype=torch.float32)
observation = Brownian_Motion_simulator1(true_params).to('cuda').unsqueeze(-1)
observation.shape

In [ ]:
samples = posterior_BM.sample((10000,), x=observation, show_progress_bars=False).cpu()

_ = analysis.pairplot(
    samples,
    limits=[[-1, 2]],
    figsize=(10, 10),
    points_offdiag={'markersize': 6},
    labels=['$\log(D)$'],
    bins=1000
)

posterior_samples = samples.cpu().numpy()

posterior_samples.mean()